# Preparing the Data


In [1]:
 # Loading the data

from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/drive')

PO_data=pd.read_csv('/content/drive/MyDrive/NUS-ISS AIS Projects/Project 1/Data/enlarged_dataset.csv')

# Display the first few rows of the dataset
print('\n First few rows of the PO Dump: \n', PO_data.head(), '\n')

Mounted at /content/drive

 First few rows of the PO Dump: 
    PO_NUM         ITEM_NAME                       PART_DESCRIPTION ITEM_CODE  \
0  100000           Gripper     Pneumatic gripper for robotic arms   MOT7793   
1  100000        3D Printer     Compact 3D printer for prototyping   MOT1546   
2  100001   Pressure Sensor         High-precision pressure sensor   KYO7240   
3  100001  Proximity Sensor             Inductive proximity sensor   KYO2884   
4  100002   Control Cabinet  Industrial control cabinet with locks   IND4956   

                         SUPPLIER_NAME SUPPLIER_CODE  ORDERED_QUANTITY  \
0  MOTHERSON SUMI WIRING INDIA LIMITED      VD904014                39   
1  MOTHERSON SUMI WIRING INDIA LIMITED      VD904014               180   
2         KYOWA MANUFACTURING CO., LTD      VD650721                67   
3         KYOWA MANUFACTURING CO., LTD      VD650721               179   
4                  INDEANA ENGINEERING      VD671527               300   

   FAULTED_PA

Calculating Delivery Time for each line item

In [2]:
#Calculating Delivery Time

# Convert 'DOC_CREATION_DATE' and 'DELIVERY_DATE' to datetime objects
PO_data['DOWNPAYMENT_DATE'] = pd.to_datetime(PO_data['DOWNPAYMENT_DATE'])
PO_data['DELIVERY_DATE'] = pd.to_datetime(PO_data['DELIVERY_DATE'])

# Calculate delivery time in days
PO_data['DELIVERY_TIME'] = (PO_data['DELIVERY_DATE'] - PO_data['DOWNPAYMENT_DATE']).dt.days

PO_data['DELIVERY_TIME'] = PO_data['DELIVERY_TIME'].abs()

# Print delivery time for each vendor
print(PO_data[['SUPPLIER_CODE', 'DELIVERY_TIME']])

      SUPPLIER_CODE  DELIVERY_TIME
0          VD904014             21
1          VD904014             38
2          VD650721             37
3          VD650721             27
4          VD671527             99
...             ...            ...
58662      VD492539             16
58663      VD492539              6
58664      VD406065             26
58665      VD994773             40
58666      VD994773             39

[58667 rows x 2 columns]


In [3]:
#Checking for improper Delivery times

#Print all vendors with negative delivery time
negative_delivery_time = PO_data[PO_data['DELIVERY_TIME'] < 0]

#Total number of vendors with negative delivery date
print('Total Number of vendors with Negative deliveries:', len(negative_delivery_time['SUPPLIER_CODE'].unique()), '\n')
print(negative_delivery_time['SUPPLIER_CODE'].value_counts())

Total Number of vendors with Negative deliveries: 0 

Series([], Name: count, dtype: int64)


In [4]:
Original_price = PO_data['PRICE']
#Original_item_value = PO_data['ITEM_VALUE']
Original_delivery_time = PO_data['DELIVERY_TIME']

# EDA

In [5]:
# observing the shape of the data
print('Shape of the Data is: \t', PO_data.shape, '\n')

Shape of the Data is: 	 (58667, 13) 



In [6]:
# Check the data types and non-null counts
print('\nData Types and Non-null Counts:\t', PO_data.info(), '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58667 entries, 0 to 58666
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   PO_NUM            58667 non-null  int64         
 1   ITEM_NAME         58667 non-null  object        
 2   PART_DESCRIPTION  58667 non-null  object        
 3   ITEM_CODE         58667 non-null  object        
 4   SUPPLIER_NAME     58667 non-null  object        
 5   SUPPLIER_CODE     58667 non-null  object        
 6   ORDERED_QUANTITY  58667 non-null  int64         
 7   FAULTED_PARTS     58667 non-null  int64         
 8   PRICE             58667 non-null  float64       
 9   PO_VALUE          58667 non-null  float64       
 10  DOWNPAYMENT_DATE  58667 non-null  datetime64[ns]
 11  DELIVERY_DATE     58667 non-null  datetime64[ns]
 12  DELIVERY_TIME     58667 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(4), object(5)
memory usage: 5.8+ MB


In [7]:
# Check for missing values
print('Missing Values: \n', PO_data.isnull().sum(), '\n')

Missing Values: 
 PO_NUM              0
ITEM_NAME           0
PART_DESCRIPTION    0
ITEM_CODE           0
SUPPLIER_NAME       0
SUPPLIER_CODE       0
ORDERED_QUANTITY    0
FAULTED_PARTS       0
PRICE               0
PO_VALUE            0
DOWNPAYMENT_DATE    0
DELIVERY_DATE       0
DELIVERY_TIME       0
dtype: int64 



In [8]:
# Check for duplicate rows
print('Number of duplicate rows: \t', PO_data.duplicated().sum(), '\n')

Number of duplicate rows: 	 0 



In [9]:
#Summary of all the numerical data
print('All the Numerical Features: \n', PO_data.select_dtypes(include=['number']).columns, '\n')

# Summary statistics for numerical features
print('Summary for Numberical Features: \n', PO_data.describe(), '\n')

All the Numerical Features: 
 Index(['PO_NUM', 'ORDERED_QUANTITY', 'FAULTED_PARTS', 'PRICE', 'PO_VALUE',
       'DELIVERY_TIME'],
      dtype='object') 

Summary for Numberical Features: 
               PO_NUM  ORDERED_QUANTITY  FAULTED_PARTS         PRICE  \
count   58667.000000      58667.000000   58667.000000  58667.000000   
mean   115010.739496        150.165408       5.705882   2563.427641   
min    100000.000000          1.000000       0.000000     41.350000   
25%    107535.000000         75.000000       0.000000   1300.845000   
50%    114965.000000        150.000000       0.000000   2575.440000   
75%    122516.500000        225.000000       3.000000   3729.720000   
max    129999.000000        300.000000     300.000000   5987.200000   
std      8657.877555         86.742675      25.070171   1454.977483   

           PO_VALUE               DOWNPAYMENT_DATE  \
count  5.866700e+04                          58667   
mean   3.847127e+05  2023-07-02 15:11:08.798472448   
min    6.

In [10]:
#Summary of all the categorical data
print('All the Categorical Features: \n', PO_data.select_dtypes(include=['object', 'category']).columns, '\n')

# Summary statistics for categorical features
print('Summary for Categorical Features: \n', PO_data.describe(include=['object']), '\n')

All the Categorical Features: 
 Index(['ITEM_NAME', 'PART_DESCRIPTION', 'ITEM_CODE', 'SUPPLIER_NAME',
       'SUPPLIER_CODE'],
      dtype='object') 

Summary for Categorical Features: 
         ITEM_NAME                            PART_DESCRIPTION ITEM_CODE  \
count       58667                                       58667     58667   
unique         15                                          15      1199   
top     VFD Drive  Variable Frequency Drive for motor control   PRA4679   
freq         4506                                        4506        81   

                           SUPPLIER_NAME SUPPLIER_CODE  
count                              58667         58667  
unique                                80            80  
top     Praja Controls & Systems Pvt Ltd      VD899831  
freq                                 828           828   



In [11]:
# Standardize the numeric columns

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
PO_data[['PRICE','DELIVERY_TIME']] = scaler.fit_transform(PO_data[['PRICE','DELIVERY_TIME']])


In [12]:
# Encode categorical variables
PO_data = pd.get_dummies(PO_data, columns=['SUPPLIER_NAME', 'ITEM_NAME'], drop_first=True)

# Training with Collaborative Fitting with KNN

In [13]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357285 sha256=76d0f550375bbcde3eff7584115fda9e85447d88e0c9782000bfd655d5548888
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


Ranking Vendors out of 10 based on Price

In [14]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Prepare data for Surprise library
reader = Reader(rating_scale=(1, 10))  # Adjust rating scale as needed
data = Dataset.load_from_df(PO_data[['SUPPLIER_CODE', 'PRICE', 'DELIVERY_TIME']], reader)

# Split data into train and test sets
trainset, testset = train_test_split(data, test_size=.25)

# Train the SVD model
algo = SVD()
algo.fit(trainset)

# Predict vendor ratings based on price
predictions = algo.test(testset)

# Store predictions in a DataFrame
prediction_df = pd.DataFrame(predictions)

# Get predicted ratings
predicted_ratings = prediction_df[['uid', 'est']]

# Group by vendor (uid) and calculate the average predicted rating
vendor_ratings = predicted_ratings.groupby('uid')['est'].mean()


# Scale the average predicted rating to be between 1 and 10
scaler = MinMaxScaler(feature_range=(1, 10))
scaled_vendor_ratings = scaler.fit_transform(vendor_ratings.values.reshape(-1, 1))

# Create a DataFrame with vendor and rating
vendor_ranking = pd.DataFrame({'SUPPLIER_CODE': vendor_ratings.index, 'Vendor_Rating': scaled_vendor_ratings.flatten()})

# Sort vendors based on their rating
vendor_ranking = vendor_ranking.sort_values('Vendor_Rating', ascending=False)

vendor_ranking

,SUPPLIER_CODE,Vendor_Rating
0,VD101149,1.0
1,VD122239,1.0
58,VD784623,1.0
57,VD780586,1.0
56,VD776359,1.0
...,...,...
25,VD424465,1.0
24,VD412248,1.0
23,VD406065,1.0
22,VD369278,1.0


Traning model for vendor recommendation based on Price


In [15]:
from surprise import KNNBasic
from surprise import accuracy
from surprise import Reader
from surprise import Dataset
from sklearn.model_selection import train_test_split

y = PO_data['PRICE']
X = PO_data.drop(columns=['PRICE'])

# Split the data (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Create a Surprise Dateset
reader = Reader(rating_scale=(PO_data['PRICE'].min(), PO_data['PRICE'].max()))

# Create the trainset from the training data
train_df = pd.DataFrame({'userID': X_train.index, 'itemID': X_train['ITEM_CODE'], 'rating': y_train})
trainset = Dataset.load_from_df(train_df[['userID', 'itemID', 'rating']], reader)

# Use the SVD algorithm
model = KNNBasic()

sim_options = {
    'name': 'cosine',  # Or 'pearson'
    'user_based': False  # True for user-based, False for item-based
}
model = KNNBasic(sim_options=sim_options)

# Train the model on the Surprise trainset
model.fit(trainset.build_full_trainset())

# Make predictions - need to create testset similar to trainset
test_df = pd.DataFrame({'userID': X_test.index, 'itemID': X_test['ITEM_CODE'], 'rating': y_test})
testset = Dataset.load_from_df(test_df[['userID', 'itemID', 'rating']], reader).build_full_trainset().build_testset() # create testset
predictions = model.test(testset) # get predictions on the testset

# Evaluate the model
accuracy.mse(predictions)

# Evaluate accuracy
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
MSE: 1.0105
RMSE: 1.0052


1.0052350210939518

In [16]:
# Replace the scaled columns with the original values
PO_data['PRICE'] = Original_price
#PO_data['ITEM_VALUE'] = Original_item_value
PO_data['DELIVERY_TIME'] = Original_delivery_time

Get top 3 Vendors for a particular part name on the basis of delivery time

In [18]:
# Get the top 3 vendors for a specific component

#randomly select a part_name
import random

# Get all available column names
available_columns = PO_data.columns

# Check if 'PART_NAME' exists, otherwise, use a different relevant column
part_name_column = 'PART_NAME' if 'PART_NAME' in available_columns else available_columns[0]  # Replace available_columns[0] with an appropriate column if needed

# Randomly select a part_name
part_name = random.choice(PO_data[part_name_column].unique())

print(f"Randomly selected part name: {part_name}")

component_id = PO_data[PO_data[part_name_column] == part_name]['ITEM_CODE'].iloc[0]

recommended_vendors = PO_data[PO_data['ITEM_CODE'] == component_id]
recommended_vendors = recommended_vendors.sort_values(by='DELIVERY_TIME', ascending=True)

# Print the top N vendors for the component
top_vendors = recommended_vendors.head(3)
print(top_vendors[['SUPPLIER_CODE', 'DELIVERY_TIME', 'PRICE']])


Randomly selected part name: 126547
      SUPPLIER_CODE  DELIVERY_TIME   PRICE
14565      VD996609              6  724.64
12759      VD996609              6  754.25
9472       VD996609              6  793.37
